In [ ]:
from pymatgen import Structure
from monty.serialization import loadfn
import os

os.chdir('/home/jinho93/test/ml/hfo2')

data = loadfn('data.json')
train_structures = [d['structure'] for d in data]
train_energies = [d['outputs']['energy'] for d in data]
train_forces = [d['outputs']['forces'] for d in data]



In [ ]:
from maml.apps.pes import SNAPotential
nnp = SNAPotential()
nnp.train(train_structures=train_structures,
          train_energies=train_energies,
          train_forces=train_forces,
          cutoff_type=1, atom_energy={'Hf': -10.14, 'O': -4.14},
          r_etas = [0.5, 1.0, 2.0],
          a_etas = [0.5, 2.0],
          r_shift=[0.0],
          zetas=[1.0, 4.0],
          epochs = 100, r_cut=5, hidden_layers=[24, 24])

In [ ]:
import numpy as np
from maml.utils import pool_from, convert_docs

train_pool = pool_from(train_structures, train_energies, train_forces)
_, df = convert_docs(train_pool)

weights = np.ones(len(df['dtype']), )

# set the weights for energy equal to 100
weights[df['dtype'] == 'energy'] = 1
weights[df['dtype'] == 'force'] = 10000

from maml.base import SKLModel
from maml.describers import BispectrumCoefficients
from sklearn.linear_model import LinearRegression
from maml.apps.pes import SNAPotential

element_profile = {'Hf': {'r': 6.0, 'w': 1}, 'O': {'r': 6.0, 'w': 1}}
describer = BispectrumCoefficients(rcutfac=0.5, twojmax=8, element_profile=element_profile, 
                                   quadratic=False, pot_fit=True)
model = SKLModel(describer=describer, model=LinearRegression())
snap = SNAPotential(model=model)
snap.train(train_structures, train_energies, train_forces, sample_weight=weights)

In [ ]:
from phonopy.interface.calculator import read_crystal_structure
unitcell, _ = read_crystal_structure("BPOSCAR", interface_mode='vasp')

In [ ]:
nnp.write_input()
nnp.write_param()

In [ ]:

from maml.apps.pes import EnergyForceStress
from pymatgen import Structure

# efs_calculator = EnergyForceStress(ff_settings=nnp)
# efs_calculator.set_lmp_exe('/home/jinho93/Downloads/n2p2/bin/lmp_mpi')

efs_calculator = EnergyForceStress(ff_settings=snap)
# efs_calculator.set_lmp_exe('/home/jinho93/Downloads/ls /bin/lmp_mpi')
efs_calculator.set_lmp_exe('/home/jinho93/Downloads/lammps-stable_29Sep2021_update2/src/lmp_intel_cpu_intelmpi')

sets_of_forces = []
structures = [Structure.from_file(f'mono/POSCAR-0{i:02d}') for i in range(1, 19)]
for e, f, s in efs_calculator.calculate(structures):
    sets_of_forces.append(f)

In [1]:

from phonopy import Phonopy
import numpy as np

from phonopy.interface.calculator import read_crystal_structure
unitcell, _ = read_crystal_structure("POSCAR", interface_mode='vasp')

phonon = Phonopy(unitcell,
                 np.identity(3) * 4,
#                  primitive_matrix=np.reshape('0.0 0.5 0.5  0.5 0.0 0.5  0.5 0.5 0.0'.split(), (3, 3)))
                  primitive_matrix=np.identity(3))

TypeError: Array shape of cell is not 3x3.

In [ ]:
phonon.generate_displacements(distance=0.01)

phonon.set_forces(sets_of_forces)

phonon.produce_force_constants()

In [ ]:
phonon._factor = phonon.get_unit_conversion_factor() * 33.4

In [ ]:
from phonopy.phonon.band_structure import get_band_qpoints
from phonopy.phonon.band_structure import get_band_qpoints_and_path_connections
path = [[[0, 0, 0], [0.0, 0, 0.5], [-0.5, 0.0, 0.5], [-0.5, 0.0, 0.5], [0, 0, 0]],
        [[0, 0, 0], [-0.5, 0.5, 0.5], [0.5, 0, 0], [0.0, 0.0, 0.0]]]
path = [[[0, 0, 0], [0, 0.5, 0.0], [0.5, 0.5, 0], [0, 0, 0]],
        [[0, 0, 0], [0.0, 0.0, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.5]]]
#labels = ["$\\Gamma$", "X", "M", "M", "$\\Gamma$", "R", "M"]
#qpoints, connections = get_band_qpoints_and_path_connections(path, npoints=51)
#phonon.run_band_structure(qpoints, path_connections=connections, labels=labels, with_eigenvectors=True)
#phonon.plot_band_structure().show()

phonon.auto_band_structure(plot=True, with_eigenvectors=True).show()

In [ ]:
phonon.write_animation([0.5,0.5,0], anime_type='v_sim')
#phonon.write_animation([0.5,0.0,0], band_index=0,amplitude=5, num_div=20, anime_type='xyz')

In [ ]:
import os
os.chdir('/home/jinho93/ml/ml-lao/6248/phon')

from phonopy import Phonopy
import numpy as np

from phonopy.interface.calculator import read_crystal_structure
unitcell, _ = read_crystal_structure("BPOSCAR", interface_mode='vasp')

phonon = Phonopy(unitcell,
                 [[2, 0, 0], [0, 2, 0], [0, 0, 2]],
                 primitive_matrix=[[0, 0.5, 0.5],
                                   [0.5, 0, 0.5],
                                   [1, 2, 1/3]])

In [ ]:
phonon.generate_displacements()
phonon.write_
supercells = phonon.supercells_with_displacements
print(supercells[0])

In [ ]:
from maml.apps.pes import EnergyForceStress

sets_of_forces = []
efs_calculator = EnergyForceStress(ff_settings=nnp)
# efs_calculator.LMP_EXE = '/home/jinho93/Downloads/n2p2/bin/lmp_mpi'
efs_calculator.set_lmp_exe('/home/jinho93/Downloads/n2p2/bin/lmp_mpi')

for s in supercells:
    struct = Structure(s.cell, s.get_chemical_symbols(), s.get_scaled_positions())
    struct.to('POSCAR', f'POSCAR-{n}')
    energy, forces, stresses = efs_calculator.calculate([struct])[0]
    sets_of_forces.append(forces)
    break

In [ ]:
efs_calculator = EnergyForceStress(ff_settings=snap)
# efs_calculator.set_lmp_exe('/home/jinho93/Downloads/ls /bin/lmp_mpi')
efs_calculator.set_lmp_exe('/home/jinho93/Downloads/lammps-stable_29Sep2021_update2/src/lmp_intel_cpu_intelmpi')
output = efs_calculator.calculate(train_structures)

In [ ]:
for i, j in zip(output, train_forces):
    print(i[1][0], j[0])

In [ ]:
output[0][1]